In [ ]:
# So Colab can use the python files...
!git clone --single-branch -b develop https://github.com/muellevin/Studienarbeit.git
%cd Studienarbeit/Detection_training

# 1. Setup Paths

In [ ]:
import os
import Tensorflow.scripts.Paths as pp
from ultralytics import YOLO

paths = pp.paths
paths.setup_paths()
CUSTOM_MODEL_NAME = 'raccoon_yolov8n_320_B16_ep34'



# 2. Download pretrained Tensorflow Model and install TFOD

In [ ]:
%pip install ultralytics

# Make the yaml file

In [ ]:
with open(pp.YOLO_CONFIG_PATH, "w") as config:
    config.write("train: ../trainset/images\n")
    config.write("test: ../testset/images\n")
    config.write("val: ../devset/images\n\n")
    config.write('nc: {}\n'.format(len(pp.LABELS)))
    config.write("names: [")
    config.write("error_fix, ")
    for i in range(0, len(pp.LABELS)):
        label = pp.LABELS[i]
        if i < len(pp.LABELS) - 1:
            config.write("'{}', ".format(label['name']))
        else:
            config.write("'{}']\n".format(label['name']))


Install TFOD (TensorFlow Object Detection)

# 4. Upload dataset to Colab

Upload the dataset to google drive

In [ ]:
GOOGLE_DRIVE_PATH = '/content/drive/MyDrive/Studienarbeit'

if os.name =='posix':
    from google.colab import drive

    drive.mount('/content/drive')

    import shutil
    shutil.copy(os.path.join(GOOGLE_DRIVE_PATH, 'yolo_' + pp.DATASET_NAME), paths.YOLO_IMG_PATH)

In [ ]:
if os.path.exists(pp.YOLO_DATASET):
  !tar -zxvf {pp.YOLO_DATASET}

In [ ]:
# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch

# Use the model
results = model.train(
    data=pp.YOLO_CONFIG_PATH,
    epochs=1,
    imgsz=320,
    batch=16,
    name=CUSTOM_MODEL_NAME)  # train the model

In [ ]:
results = model.val()  # evaluate model performance on the validation set

# Zip it and save it in Google Drive

In [ ]:
import shutil
shutil.make_archive(os.path.join(GOOGLE_DRIVE_PATH, CUSTOM_MODEL_NAME), 'zip', "runs")

# Test detections on images

The loss metrics and other stuff is already saved in the zip file as .png

In [ ]:
# Load the trained model
model = YOLO(os.path.join(paths.MODEL_PATH, CUSTOM_MODEL_NAME, "weights", "best.pt"))

In [ ]:
# Validate model locally
results = model.val(data=pp.YOLO_CONFIG_PATH,)  # evaluate model performance on the validation set

In [ ]:
# Run on a few pictures and see how it goes.
results = model(source=os.path.join(paths.DEVSET_PATH), show=True, conf=0.3, save=True)
